In [1]:
import os
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification\\research'

In [2]:
os.chdir("../")
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [10]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [11]:
#03_prepare_callbacks

 #Initialising the entity
from dataclasses import dataclass
from pathlib import Path
# src/cnnClassifier/configuration.py
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import PrepareCallbacksConfig

import os
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])
        
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config['prepare_callbacks']
        model_ckpt_dir = os.path.dirname(config['checkpoint_model_filepath'])
        create_directories([
            Path(model_ckpt_dir),
            Path(config['tensorboard_root_log_dir'])
        ])
        
        return PrepareCallbacksConfig(
            root_dir=Path(config['root_dir']),
            tensorboard_root_log_dir=Path(config['tensorboard_root_log_dir']),
            checkpoint_model_filepath=Path(config['checkpoint_model_filepath'])
        )
  
#Creating Components

import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
#from cnnClassifier.entity.config_entity import PrepareCallbacksConfig

class PrepareCallback_:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        checkpoint_filepath = self.config.checkpoint_model_filepath
        # Ensure the filepath ends with .keras
        if not checkpoint_filepath.suffix == '.keras':
            checkpoint_filepath = checkpoint_filepath.with_suffix('.keras')

        return tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_best_only=True
        )

        
    @property   
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

#Pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback_(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks  # Remove parentheses

except Exception as e:
    raise e

[2024-05-24 23:17:19,875: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-24 23:17:19,892: INFO: common: yaml file: config\params.yaml loaded successfully]
[2024-05-24 23:17:19,895: INFO: common: created directories at: artifacts]
[2024-05-24 23:17:19,897: INFO: common: created directories at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-05-24 23:17:19,899: INFO: common: created directories at: artifacts\prepare_callbacks\tensorboard_log_dir]
